In [1]:
# RSI PowerZone Weekly
import indicator
import numpy as np
import pandas as pd
import price
import trade_statistics
from bokeh.plotting import figure, show

In [2]:
"""Trade Parameters."""
exchange = "LSE"
tidm = "RR."
filename = f"{exchange}_{tidm}_prices.csv"
history_start = "2003-03-19"
history_end = "2021-08-31"
p_sma = 40  # Simple Moving Average (SMA) look back period.
p_rsi = 4  # Relative Strength Index (RSI) look back period.
pz1 = 30  # First RSI PowerZone entry level.
pz2 = 25  # Second RSI PowerZone entry level.
pze = 75  # RSI PowerZone exit level.
position_size = 10000  # Position size in major currency unit.
charges = 6  # Commission per trade (round trip).
stamp_duty = 0.5  # Stamp Duty percentage.
stop_loss = 20  # Stop Loss percentage.

In [3]:
def state_signal(entry_signal, exit_signal, period):
    """Calculate trade state signals."""
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ["entry", "exit"]
    df["state"] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], "entry"] == 1 and df.loc[df.index[i - 1], "state"] == 0:
            df.loc[df.index[i], "state"] = 1
        elif df.loc[df.index[i], "exit"] == 1:
            df.loc[df.index[i], "state"] = 0
        else:
            df.loc[df.index[i], "state"] = df.loc[df.index[i - 1], "state"]
    return df.state


def trade_list(close, entry_flag, exit_flag):
    """Generate trade list."""
    ref = pd.concat([close, entry_flag, exit_flag], axis=1)
    ref.columns = ["close", "entry_flag", "exit_flag"]
    df = pd.DataFrame(columns=["entry_price", "exit_date", "exit_price"])
    df.entry_price = ref.close[ref.entry_flag == 1]
    df.exit_date = ref.index[ref.exit_flag == 1]
    df.exit_price = ref.close[ref.exit_flag == 1].values
    return df

In [4]:
"""Import Price Data."""

# Import weekly closing prices.
df = price.weekly_close(exchange, tidm, history_start, history_end)

In [5]:
"""Calculate Indicators."""

# Simple moving average (SMA).
df["sma"] = df.close.rolling(p_sma).mean()
sma = df.sma

# Relative Strength Index (RSI).
df["rsi"] = indicator.rsi(df.close, p_rsi)

In [6]:
# Trade system logic.

# Check closing price is above SMA.
df["trend"] = np.where(df.close > df.sma, 1, 0)

# Check for RSI cross below first PowerZone.
df["pz1"] = np.where(np.logical_and(df.rsi.shift(periods=1) >= pz1, df.rsi < pz1), 1, 0)

# Check whether RSI is below second PowerZone and price is above SMA.
df["pz2"] = np.where(np.logical_and(df.trend == 1, df.rsi < pz2), 1, 0)

# Check for RSI cross above exit level.
df["pze"] = np.where(np.logical_and(df.rsi.shift(periods=1) < pze, df.rsi >= pze), 1, 0)

# Trade #1 entry signal.
df["sig1"] = np.where(np.logical_and(df.trend == 1, df.pz1 == 1), 1, 0)

# Trade #1 state signal.
df["ss1"] = state_signal(df.sig1, df.pze, p_sma)

# Trade #2 entry signal.
df["sig2"] = np.where(np.logical_and(df.ss1.shift(periods=1) == 1, df.pz2 == 1), 1, 0)

# Trade #2 state signal.
df["ss2"] = state_signal(df.sig2, df.pze, p_sma)

# Trade entry & exit flags.
df["en1"] = np.where(np.logical_and(df.ss1 == 1, df.ss1.shift(periods=1) == 0), 1, 0)
df["ex1"] = np.where(np.logical_and(df.ss1 == 0, df.ss1.shift(periods=1) == 1), 1, 0)
df["en2"] = np.where(np.logical_and(df.ss2 == 1, df.ss2.shift(periods=1) == 0), 1, 0)
df["ex2"] = np.where(np.logical_and(df.ss2 == 0, df.ss2.shift(periods=1) == 1), 1, 0)

In [7]:
# Trade list based on RSI exit only (no Stop Loss).

# Trade List #1.
td1 = trade_list(df.close, df.en1, df.ex1)

# Trade List #2.
td2 = trade_list(df.close, df.en2, df.ex2)

# Create full trade list.
td = pd.concat([td1, td2])

# Sort trades in ascending date order.
td = td.sort_index()

# Append trade percentage price change to trade list.
td["chg_pct"] = ((td.exit_price - td.entry_price) / td.entry_price) * 100

In [8]:
# Trade list incorporating Stop Loss.
tdm = pd.DataFrame(columns=["entry_price", "exit_date", "exit_price"])
tdm.entry_price = td.entry_price
tdm.exit_date = td.exit_date
tdm.exit_price = td.exit_price

for i in range(0, len(td)):
    # Loop through trade list based on RSI exit and check if stop loss is triggered.
    ps = pd.DataFrame(columns=["close", "stop", "exit"])
    ps.close = df.close.loc[td.index[i] : td.exit_date[i]]
    ps.stop = ps.close[0] * (1 - (stop_loss / 100))
    ps.exit = np.where(ps.close < ps.stop, 1, 0)

    # Create modified trade list incorporating Stop Loss.
    if ps.exit.sum() > 0:
        exit_date = ps.exit.idxmax(axis=1, skipna=True)
        tdm.loc[tdm.index[i], "exit_date"] = exit_date
        tdm.loc[tdm.index[i], "exit_price"] = ps.close.loc[exit_date]
    else:
        tdm.loc[tdm.index[i], "exit_date"] = td.loc[td.index[i], "exit_date"]
        tdm.loc[tdm.index[i], "exit_price"] = td.loc[td.index[i], "exit_price"]

In [9]:
# Cumulative profit after costs.
tdm["weeks"] = (tdm.exit_date - tdm.index) / np.timedelta64(1, "W")
tdm["chg_pct"] = ((tdm.exit_price - tdm.entry_price) / tdm.entry_price) * 100
tdm["charges"] = charges
tdm["stamp_duty"] = position_size * (stamp_duty / 100)
tdm["profit"] = position_size * (tdm.chg_pct / 100) - tdm.charges - tdm.stamp_duty
tdm["cum_profit"] = tdm.profit.cumsum()
cum_profit = tdm.cum_profit
tdm.round(2)

,entry_price,exit_date,exit_price,weeks,chg_pct,charges,stamp_duty,profit,cum_profit
date,,,,,,,,,
2004-05-14,0.75,2004-06-25,0.83,6.0,11.01,6,50.0,1044.91,1044.91
2004-05-21,0.74,2004-06-25,0.83,5.0,12.95,6,50.0,1239.22,2284.13
2004-08-13,0.78,2004-09-17,0.86,5.0,10.09,6,50.0,952.79,3236.92
2004-08-20,0.78,2004-09-17,0.86,4.0,10.33,6,50.0,976.91,4213.83
2004-12-10,0.87,2005-01-28,0.90,7.0,3.15,6,50.0,258.92,4472.75
2004-12-17,0.86,2005-01-28,0.90,6.0,4.80,6,50.0,424.00,4896.75
2005-03-18,0.86,2005-05-20,0.93,9.0,8.48,6,50.0,791.54,5688.29
2006-05-12,1.52,2006-07-28,1.53,11.0,0.85,6,50.0,28.81,5717.10
2006-05-19,1.42,2006-07-28,1.53,10.0,8.04,6,50.0,747.58,6464.67


In [10]:
# Trade Statistics
ts = trade_statistics.summary(tidm, sma, position_size, tdm)
ts

TIDM                               RR.
Start Date                  2003-12-19
End Date                    2021-08-27
Analysis Years                    17.7
Position Size                10,000.00
Net Profit                   10,632.95
Annual %                           4.2
Charges                         126.00
Stamp Duty                    1,050.00
Total Trades                        21
Winning Trades                      19
Losing Trades                        2
Winning %                         90.5
Trades per Year                    1.2
Average Profit %                   5.6
Average Profit                  506.33
Average Weeks                     10.3
Average Winning Profit %           7.8
Average Winning Profit          724.12
Average Winning Weeks             10.4
Average Losing Profit %          -15.1
Average Losing Profit        -1,562.65
Average Losing Weeks               9.5
dtype: object

In [11]:
x = tdm.index
y = tdm.cum_profit
p = figure(title="Simple line example", x_axis_type="datetime", x_axis_label="x", y_axis_label="y",)
p.line(x, y, legend_label="Temp.", line_width=2)
show(p)